In [1]:
import gym
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from gym import wrappers
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.optimizers import SGD, RMSprop, Adam, Adamax

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def plot_running_avg(totalrewards):
  N = len(totalrewards)
  running_avg = np.empty(N)
  for t in range(N):
    running_avg[t] = totalrewards[max(0, t-100):(t+1)].mean()
  plt.plot(running_avg)
  plt.title("Running Average")
  plt.show()

In [3]:
env = gym.make('LunarLander-v2')


In [4]:
# build a set of samples so we can get a scaler fitted.
observation_samples = []

n_samples = 1000
# play a bunch of games randomly and collect observations
done = False
for n in range(n_samples):
    observation = env.reset()
    observation_samples.append(observation)
    done = False
    while not done:
        action = np.random.randint(0, env.action_space.n)
        observation, reward, done, _ = env.step(action)
        observation_samples.append(observation)
    #print(len(observation_samples))
    

observation_samples = np.array(observation_samples)

In [5]:
#env = wrappers.Monitor(env, 'monitor-folder', force=True)


In [6]:
scaler = StandardScaler()
scaler.fit(observation_samples)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:

def create_nn():
    model = Sequential()
    model.add(Dense(128, init='lecun_uniform', input_shape=(8,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.3)) #I'm not using dropout, but maybe you wanna give it a try?

    model.add(Dense(256, init='lecun_uniform'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))

    model.add(Dense(1, init='lecun_uniform'))
    model.add(Activation('linear')) #linear output so we can have range of real-valued outputs

#     rms = RMSprop(lr=0.005)
#     sgd = SGD(lr=0.1, decay=0.0, momentum=0.0, nesterov=False)
# try "adam"
#     adam = Adam(lr=0.0005)
    adamax = Adamax() #Adamax(lr=0.001)
    model.compile(loss='mse', optimizer=adamax)
#     model.summary()
    return model

In [8]:
class Model:
  def __init__(self, env, scaler):
    self.env = env
    self.scaler = scaler
    self.models = []
    for i in range(env.action_space.n):
        model = create_nn()  # one nn per action
        self.models.append(model) 

  def predict(self, s):
    X = self.scaler.transform(np.atleast_2d(s))
    return np.array([m.predict(np.array(X), verbose=0)[0] for m in self.models])

  def update(self, s, a, G):
    X = self.scaler.transform(np.atleast_2d(s))
    self.models[a].fit(np.array(X), np.array([G]), nb_epoch=1, verbose=0)

  def sample_action(self, s, eps):
    if np.random.random() < eps:
      return self.env.action_space.sample()
    else:
      return np.argmax(self.predict(s))

In [9]:
def play_one(env, model, eps, gamma):
  observation = env.reset()
  done = False
  full_reward_received = False
  totalreward = 0
  iters = 0
  while not done:
    action = model.sample_action(observation, eps)
    prev_observation = observation
    observation, reward, done, info = env.step(action)
            
    # update the model
    # standard Q learning TD(0)
    next = model.predict(observation)
    G = reward + gamma*np.max(next)
    model.update(prev_observation, action, G)
    totalreward += reward
    iters += 1
    
  return totalreward, iters

In [10]:
model = Model(env, scaler)
gamma = 0.99

N = 8096
totalrewards = np.empty(N)
costs = np.empty(N)
for n in range(N):
    eps = 1.0/np.sqrt(n+1)
    totalreward, iters = play_one(env, model, eps, gamma)
    totalrewards[n] = totalreward
    if n % 10 == 0:
      print("episode:", n, "iters", iters, "total reward:", totalreward, "eps:", eps, "avg reward (last 100):", totalrewards[max(0, n-100):(n+1)].mean())
    if totalrewards[max(0, n-100):(n+1)].mean() >= 200:
        break

print("avg reward for last 100 episodes:", totalrewards[-100:].mean())
print("total steps:", totalrewards.sum())

plt.plot(totalrewards)
plt.title("Rewards")
plt.show()

plot_running_avg(totalrewards)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(8,), kernel_initializer="lecun_uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="lecun_uniform")`
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="lecun_uniform")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


episode: 0 iters 60 total reward: -165.6311518075022 eps: 1.0 avg reward (last 100): -165.6311518075022
episode: 10 iters 197 total reward: -407.7857820092338 eps: 0.30151134457776363 avg reward (last 100): -262.9917306778315
episode: 20 iters 94 total reward: -298.5203540576749 eps: 0.2182178902359924 avg reward (last 100): -277.8942768046882
episode: 30 iters 154 total reward: -383.49196224087484 eps: 0.1796053020267749 avg reward (last 100): -280.14180346492054
episode: 40 iters 97 total reward: -386.8394693824064 eps: 0.15617376188860607 avg reward (last 100): -279.34270303383346
episode: 50 iters 124 total reward: -280.1446976900187 eps: 0.14002800840280097 avg reward (last 100): -265.7952385989979
episode: 60 iters 139 total reward: -166.44324954154183 eps: 0.12803687993289598 avg reward (last 100): -257.55177523363017
episode: 70 iters 131 total reward: -384.56766960570286 eps: 0.11867816581938533 avg reward (last 100): -258.0071817405907
episode: 80 iters 87 total reward: -294.

KeyboardInterrupt: 